In [4]:
# Import modules

import math
import os
# Watch out for assert_frame_equal, independent imported
import pandas_datareader as web
from pandas.util.testing import assert_frame_equal
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from datetime import date, datetime
import scipy

/home/denny/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
Using TensorFlow backend.


NotImplementedError: data_source='GOOG' is not implemented

In [5]:
# Retrieve the information of the desired stocks
def get_stock_data(stocktype, source, start='2012-01-01', end=date.today()):
    df = web.DataReader(stocktype, source, start=start, end=end)
    df = df.sort_index(axis=0 ,ascending=True)
    return df

# df = get_stock_data('F', 'yahoo')
# df

def create_training_set(data, training_data_len, history_days):
    # Create scaled training dataset'
    train_data = data[0:training_data_len, :]

    # Split the data into x_train and y_train (x consists of history and y of following close index)
    x_train, y_train = [], []

    # Create a dataset of the desired amount of days used to predict the next day
    for i in range(history_days, len(train_data)):
        x_train.append(train_data[i-history_days:i, 0])
        y_train.append(train_data[i, 0])

    # Convert training arrays to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data so it fits the LSTM network requirements
    features = 1

    x_train = np.reshape(x_train, (x_train.shape[0], history_days, features))
    return x_train, y_train

def test_network(pred_list, real_list):
    min_confidence = 0.99
    reward_list = []
    reward = 1000
    it = 0
    real_value = real_list[0]

    for i in range(1, len(pred_list)-1):
        if pred_list[i] * min_confidence > real_value:
            margin = reward * ((real_list[i] - real_value)/real_value)
            reward = reward + margin
            reward_list.append(margin)
            it += 1
        elif pred_list[i] < real_value * min_confidence:
            margin = reward * ((real_value - real_list[i])/real_value)
            reward = reward + margin
            it += 1
            reward_list.append(margin)
        real_value = real_list[i]

#     print('Total budget after {} days = {}'.format(i+2, reward))
#     print('Total amount of investments = {}'.format(it))
    return reward


In [8]:
df = get_stock_data('GOOG', 'yahoo', start='2020-06-29')
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-06-29,1395.599976,1347.010010,1358.180054,1394.969971,1810200,1394.969971
2020-06-30,1418.650024,1383.959961,1390.439941,1413.609985,2042400,1413.609985
2020-07-01,1443.000000,1409.819946,1411.099976,1438.040039,1775200,1438.040039
2020-07-02,1482.949951,1446.420044,1446.939941,1464.699951,1859100,1464.699951


In [16]:
# RUN ALGORITHM 20 X

df = get_stock_data('VZ', 'yahoo')
history_days = 60
outcomes = []

for i in range(20):
    # Create a new data frame with only 'Close' column
    data = df.filter(['Close'])
    dataset = data.values
    training_data_len = math.ceil(len(dataset)*0.8)
    
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    
    x_train, y_train = create_training_set(scaled_data, training_data_len, 60)
    
    # Build Model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(history_days, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=1, epochs=1)    
    
    # Create new array containing scaled values from last indices
    test_data = scaled_data[training_data_len-history_days: , :]

    # Create datasets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    # Convert test set to numpy array
    x_test = np.array(x_test)

    # Reshape data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    
    # Split Data
    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions
    
    reward = test_network(np.array(valid['Predictions']), np.array(valid['Close']))
    outcomes.append(reward)

1652/1652 [==============================] - 54s 33ms/step - loss: 0.0026


/home/denny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1652/1652 [==============================] - 52s 32ms/step - loss: 0.0026


/home/denny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1652/1652 [==============================] - 49s 30ms/step - loss: 0.0034


...

/home/denny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1652/1652 [==============================] - 44s 26ms/step - loss: 0.0026


/home/denny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1652/1652 [==============================] - 45s 28ms/step - loss: 0.0019


/home/denny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
print(outcomes)

[1471.0214264049678, 1416.1439584856093, 1348.1664089754759, 1425.313145756512, 1569.4864204122496, 1598.5391694468856, 1457.2799330298699, 1530.5046859987037, 1441.1925934138205, 1561.6111310293568, 1558.5643236958226, 1567.474423420168, 1240.7132231902106, 1436.783024218241, 1485.515125516425, 1248.7425012489168, 1537.4499366719351, 1586.2754520047279, 1656.2880348621966, 1224.8256875003776]


In [ ]:
# Visualize closing price history
plt.figure(figsize=(16,8))
plt.title('AEX Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date')
plt.ylabel('Close Index')
plt.show()

In [ ]:
'''Split a part of the dataset into train and test set'''

# Create a new data frame with only 'Close' column
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset)*0.8)
training_data_len

First the value needs to be normalized in order to see which aspects were the most influential. In de praktijk is het bijna nooit mogelijk om data in een neural network te gooien zonder het eerst te preprocessen

In [ ]:
'''Scale the data'''
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

De data moet gereshaped worden omdat het LSTM netwerk verwacht dat er een 3D dataset wordt ingegeven.

In [ ]:
'''Create Training set'''

def create_training_set(data, training_data_len, history_days):
    # Create scaled training dataset'
    train_data = data[0:training_data_len, :]

    # Split the data into x_train and y_train (x consists of history and y of following close index)
    x_train, y_train = [], []

    # Create a dataset of the desired amount of days used to predict the next day
    for i in range(history_days, len(train_data)):
        x_train.append(train_data[i-history_days:i, 0])
        y_train.append(train_data[i, 0])

    # Convert training arrays to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data so it fits the LSTM network requirements
    features = 1

    x_train = np.reshape(x_train, (x_train.shape[0], history_days, features))
    return x_train, y_train

history_days = 60
x_train, y_train = create_training_set(scaled_data, training_data_len, 60)

In [ ]:
'''Build the LSTM Sequential Model'''

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(history_days, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
'''Compile and train the model'''

# Compile Model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train Model
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
'''Create test dataset'''

# Create new array containing scaled values from last indices
test_data = scaled_data[training_data_len-history_days: , :]

# Create datasets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
# Convert test set to numpy array
x_test = np.array(x_test)

# Reshape data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions-y_test)**2)
rmse

In [ ]:
# Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

# Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Data')
plt.ylabel('Close Index')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='upper left')
plt.show()

The cells below test if the neural network is able to predict the stock prices accurate enough to make profit. It will start trading with a budget of 1000 and only buys if de divergence is at least 1%. Another testing method is to check if there is a positive correlation between the predicted change of stock price and the actual change of stock price.

In [ ]:
'''Test if profit would have been made over the testing results'''
 
def test_network(pred_list, real_list):
    min_confidence = 0.99
    reward_list = []
    reward = 1000
    it = 0
    real_value = real_list[0]

    for i in range(1, len(pred_list)-1):
        if pred_list[i] * min_confidence > real_value:
            margin = reward * ((real_list[i] - real_value)/real_value)
            reward = reward + margin
            reward_list.append(margin)
            it += 1
        elif pred_list[i] < real_value * min_confidence:
            margin = reward * ((real_value - real_list[i])/real_value)
            reward = reward + margin
            it += 1
            reward_list.append(margin)
        real_value = real_list[i]

    print('Total budget after {} days = {}'.format(i+2, reward))
    print('Total amount of investments = {}'.format(it))
    return reward

reward = test_network(np.array(valid['Predictions']), np.array(valid['Close']))

In [ ]:
def test_cor(pred_list, real_list):
    rchanges, pchanges = [], []
    for i in range(1, len(pred_list)):
        real_change = real_list[i] - real_list[i-1]
        pred_change = pred_list[i] - real_list[i-1]
        rchanges.append(real_change)
        pchanges.append(pred_change)
    return rchanges, pchanges
        
r, p = test_cor(list(valid['Predictions']), list(valid['Close']))
correlation, p_value = scipy.stats.pearsonr(r, p)
print(correlation*100)


In [ ]:
'''Example of predicting a single day'''

# WERKT WSS NIET VANWEGE NIET OPNIEUW INGEVEN VAN DATA

# Create new df
new_df = df.filter(['Close'])
print(new_df[-61:-1])


# Get the last 60 day closing index values and convert the dataframe to an array
last_60_days = new_df[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)

# Create an empty list
X_test = []

# Append past 60 days
X_test.append(last_60_days_scaled)

# Convert X_test dataset to numpy array
X_test = np.array(X_test)

# Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Get predicted scaled price
pred_price = model.predict(X_test)

In [ ]:
'''Retrieve real time stock values with free version of Alpha Vantage'''

import pandas_datareader.data as web
from alpha_vantage.timeseries import TimeSeries
import sys
import random

def get_realtime(ticker, interval):
    ticker = ticker
    lines = open('av/keys').read().splitlines()
    key = random.choice(lines)
    print(key)
    time = TimeSeries(key=key, output_format='pandas')
    data = time.get_intraday(symbol=ticker, interval=interval, outputsize='full')

    print(ticker)
    print(data)

get_realtime('TSLA', '1min')

# web.get_quote_av(["AAPL", "TSLA"])

# f = web.DataReader("AAPL", "av-daily", start=datetime(2020, 2, 9),
#                     end=datetime(2020, 5, 24),
#                     api_key=os.getenv('YT5YAPB532G3OSZE'))

# f.loc["2017-02-09"]
